In [0]:
import tweepy
import pandas as pd
import numbers
from tqdm import tqdm
import re
import os.path
import datetime
from tweepy import OAuthHandler, Stream, StreamListener
import datetime
import time

pd.options.display.max_colwidth = 400
pd.options.display.max_rows = 25
pd.options.display.max_columns = None

# DATA INGESTION

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Autenticazione


In [0]:
auth = tweepy.OAuthHandler("X" , "X" )
auth.set_access_token("X", "X" )
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

## Download in batch dei tweet da twitter

N.B. All'interno si definiscono tutti i parametri che servono per il download e le feature raccolte insieme al testo del tweet (il parametro *q* viene cambiato ogni volta che si fa la raccolta con #CheccoZalone o #ToloTolo)

In [0]:
import csv
import pprint
filename = 'twitter_data_analysis'+(datetime.datetime.now().strftime("%Y-%m-%d-%H"))+'.csv'
out = []
for tweet in tweepy.Cursor(api.search, q="#CheccoZalone", lang = 'it', count=100,since="2019-12-27", tweet_mode = 'extended', wait_on_rate_limit=True, wait_on_rate_limit_notify=True).items():
    tweet_dict=json.loads(json.dumps(tweet._json))
    if ('retweeted_status' not in tweet_dict.keys() and tweet_dict['is_quote_status']==False):
      ttype = "ORIGINAL"
      out.append([tweet.id, tweet.created_at, tweet.retweet_count, tweet._json["user"]["screen_name"], tweet.full_text.encode('utf-8').decode('utf-8'), None, ttype])  
    else:
      if ('retweeted_status' in tweet_dict):
        ttype = "RT_NOQUOTE"
      else:
        ttype = "RT_QUOTE"
    out.append([tweet.id, tweet.created_at, tweet.retweet_count, tweet._json["user"]["screen_name"], tweet.full_text.encode('utf-8').decode('utf-8'), tweet.in_reply_to_screen_name,ttype])
       

bdata = pd.DataFrame(out,columns="id,time_tweet,retweet_count,user_screen_name,text,retweet_screen_name,type".split(","))

Si corregge il formato data per ottenere quello desiderato (ora e data separati)

In [0]:
from datetime import datetime

data = []
ora = []
for i in range(len(bdata)):
  timestamp = bdata["time_tweet"][i]
  date_time_obj = datetime.strptime(str(timestamp), '%Y-%m-%d %H:%M:%S')

  data.append(date_time_obj.strftime('%Y-%m-%d'))
  ora.append(date_time_obj.strftime('%H:%M:%S'))

bdata['time_tweet'] = ora
bdata['date_tweet'] = data

new_order = [0,1,7,2,3,4,5,6]
bdata = pd.DataFrame(bdata[bdata.columns[new_order]])

## Salvataggio tweet
N.B. operazione effettuata ogni volta in cui è avvenuta la raccolta

In [0]:
#esempio salvataggio

bdata = tweet_dataframe
tweet_dataframe.to_csv('checco-03-01-corretto.csv')